In [ ]:
import PathConfig
from slice_loader import SliceLoader
from read_write_mrc import *
import numpy as np
import torch
import os
import glob
import torch.nn as nn
from pytorch_network import UNet, UNetDeep
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
from utils import *
import cupy as cp


In [ ]:
##Cropping- AntiPadding Code to make Predicitons in Original GT size.

PredPath = sorted(glob.glob(os.path.join('../Data/Predictions/FinalPredictionMito+NucleusAt(0.6)/','*.mrc')))
maskFiles = sorted(glob.glob(os.path.join('../Data/User_Raw_Data/MaskTest','*.mrc'))) ## Here you have to add the Original size mask GT files inside the MaskTest folder similar to the prediction images.
outputPath = '../Data/Predictions/CroppedPredictions/'
os.makedirs(outputPath,exist_ok=True)
for i,j in zip(PredPath, maskFiles):
    img = read_mrc(i).get()
    mask = read_mrc(j).get()
    print(os.path.basename(i),os.path.basename(j))
    cropped_img = anti_padding(img,mask.shape)
    write_mrc(outputPath + os.path.basename(i),cropped_img)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(input_shape=(1,64,704,704), num_classes=2)
model = nn.DataParallel(model)
checkpoint = torch.load('Trained_Model')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

In [4]:
test_data = SliceLoader('../Data/Training_Slices/','mrc_test_Slices', 'mask_test_Slices/') 
test_loader = DataLoader(test_data , batch_size = 1 , shuffle = False)

In [5]:
len(test_loader)

189

In [6]:
predictionDirPath = '../Data/Predictions/Nucleus_Prediction'
predictionDirPath1 = '../Data/Predictions/Mitochondira_Prediction'
os.makedirs(predictionDirPath , exist_ok= True)
os.makedirs(predictionDirPath1 , exist_ok= True)

for i, (img,mask) in enumerate(test_loader):
    prediction = model(img)
    prediction = torch.sigmoid(prediction)
    prediction = prediction.detach().cpu().numpy()
    prediction1 = prediction[0, 0, :, :, :]
    prediction2 = prediction[0, 1, :, :, :]
    filename1 = os.path.join(predictionDirPath, os.path.basename(test_data.labels_list[i]))
    filename2 = os.path.join(predictionDirPath1, os.path.basename(test_data.labels_list[i]))    
    write_mrc(filename1, prediction1)
    write_mrc(filename2, prediction2)
    print(i+1, end='\r')

In [4]:
import glob,os,gc
from utils import *

maskList = sorted(glob.glob(os.path.join('../Data/Preprocessed_Data/mrc_test/*.mrc')))
nucleusList = sorted(glob.glob(os.path.join('../Data/Predictions/PredictionMerged_Nucleus/','*.mrc')))
mitoList = sorted(glob.glob(os.path.join('../Data/Predictions/PredictionMerged_Mitochondria','*.mrc')))
path = '../Data/Predictions/FinalPredictionMito+NucleusAt(0.6)/' # Can be calculated at different thresholds.
os.makedirs(path,exist_ok=True)

In [5]:
for mask,nucleusPath,mitoPath in zip(maskList, nucleusList, mitoList): 
    original_label = read_mrc(mask).get()
    merged_img_nucleus = read_mrc(nucleusPath).get()
    merged_img_mitochondria = read_mrc(mitoPath).get()
    nucleus = np.where(merged_img_nucleus<0.6, 0 , merged_img_nucleus)
    mitochondria = np.where(merged_img_mitochondria<0.6, 0 , merged_img_mitochondria)
    final_prediction = np.zeros(original_label.shape , dtype = np.int8)
    final_prediction[original_label>0] = 1
    np.place(final_prediction , nucleus>mitochondria , [2.0])
    np.place(final_prediction , mitochondria>nucleus , [5.0])
    name = os.path.basename(nucleusPath)
    write_mrc(os.path.join(path,name),final_prediction)
    del nucleus, mitochondria,final_prediction,original_label, name, merged_img_mitochondria, merged_img_nucleus
    gc.collect()
    print(f'---Done---')

---Done---
---Done---
---Done---
